In [18]:
import pandas as pd
import os
import numpy as np
import torch
import random
import pickle
from tqdm import tqdm
from collections import Counter
from torch.utils.data import DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F

In [19]:
class LoadData():
    def __init__(self,csv_file):
        self.df = pd.read_csv(os.path.join(os.getcwd(),csv_file))
        self.train_df = None
        self.test_df = None
    def load_data(self):
        self.df.columns = ['sentence','sentiment']
        self.train_df = self.df[self.df['sentiment']=='positive']
        self.test_df = self.df[self.df['sentiment']=='negative']
        self.train_df.loc[self.train_df['sentiment']=='positive','polarity'] = 1
        self.test_df.loc[self.test_df['sentiment']=='negative','polarity'] = 0

In [20]:
loaddata_obj = LoadData("imdb_dataset_small.csv") 
loaddata_obj.load_data()
train = loaddata_obj.train_df
test = loaddata_obj.test_df

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [21]:
train.head()

,sentence,sentiment,polarity
0,One of the other reviewers has mentioned that ...,positive,1.0
1,A wonderful little production. <br /><br />The...,positive,1.0
2,I thought this was a wonderful way to spend ti...,positive,1.0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1.0
5,"Probably my all-time favorite movie, a story o...",positive,1.0


In [25]:
def Corpus_Extr(df):
    print('Construct Corpus...')
    corpus = []
    sentences = df['sentence'].values
    
    for i in range(len(sentences)):
        corpus += sentences[i].lower().split()
    print('Convert Corpus to Integers')
    vocab_to_int = {word: idx for idx,word in enumerate(corpus)}
    print('Convert Phrase to Integers')
    phrase_to_int = []
    for i in range(len(sentences)):
        phrase_to_int.append([vocab_to_int[word] for word in sentences[i].lower().split()])
    return corpus,vocab_to_int,phrase_to_int
corpus,vocab_to_int,phrase_to_int = Corpus_Extr(train)

Construct Corpus...
Convert Corpus to Integers
Convert Phrase to Integers


In [26]:
def Pad_sequences(phrase_to_int,seq_length):
    pad_sequences = np.zeros((len(phrase_to_int), seq_length),dtype=int)
    for idx,row in tqdm(enumerate(phrase_to_int),total=len(phrase_to_int)):
        pad_sequences[idx, :len(row)] = np.array(row)[:seq_length]
    return pad_sequences

pad_sequences = Pad_sequences(phrase_to_int,30)

100%|██████████| 9903/9903 [00:00<00:00, 42704.28it/s]


In [10]:
class PhraseDataset(Dataset):
    def __init__(self,df,pad_sequences):
        super().__init__()
        self.df = df
        self.pad_sequences = pad_sequences
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        if 'Sentiment' in self.df.columns:
            label = self.df['Sentiment'].values[idx]
            item = self.pad_sequences[idx]
            return item,label
        else:
            item = self.pad_sequences[idx]
            return item

In [ ]:
predict_sentiment(model, "This film is terrible")

In [ ]:
predict_sentiment(model, "This film is great")